# Similarity search

In [37]:
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import qgrid

import sys
sys.path.insert(0, "..")

### Read preprocessed text

In [38]:
df = pd.read_csv("cleaned_df.csv")
df = df[["Unnamed: 0", "Unnamed: 0_x", 'text_clean', "text_no_point", 'article_name', 'count']]
df.dropna(inplace=True)
df = df.sort_values(by=['count'], ascending=False)

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_df.csv'

In [39]:
# Read labelled test df
#df.to_csv("merge.csv")
label = df[:300].copy()
label = label[['text_clean', "Unnamed: 0"]]
label.rename(columns={'text_clean': 'text', 'Unnamed: 0': 'index'}, inplace=True)
label.to_csv("to_annotate.csv")

NameError: name 'df' is not defined

### After manually annotating text

In [41]:
train = pd.read_csv("result.csv")

### TF-IDF similarity

In [42]:
# Spliting into X & y
#X = train.iloc[:, 2].values # to get th
X = train["text_wop_clean"].values # to get th

# Convert label to numeric
cleanup_label = {"sentiment": {"Ja": 1, "Nee": 0}}
train.replace(cleanup_label, inplace=True)
y = train.sentiment.values

In [43]:
# Split train and validation
from sklearn.model_selection import train_test_split
xtrain, xvalid, ytrain, yvalid = train_test_split(X, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [44]:
# Create TF-IDF
tfv = TfidfVectorizer(min_df=3, max_features=None,
                      strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
                      ngram_range=(1, 2), use_idf=1, smooth_idf=1, sublinear_tf=1)

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

#X_arr = td.fit_transform(X).toarray()

TfidfVectorizer(min_df=3, ngram_range=(1, 2), smooth_idf=1,
                strip_accents='unicode', sublinear_tf=1,
                token_pattern='\\w{1,}', use_idf=1)

In [50]:
xtrain_tfv.shape

(61, 1655)

In [12]:
# Training the classifier & predicting on test data
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
yvalid_pred = clf.predict(xvalid_tfv)

In [13]:
# Classification metrics
from sklearn.metrics import accuracy_score, classification_report
classification_report = classification_report(yvalid, yvalid_pred)

print('\n Accuracy: ', accuracy_score(yvalid, yvalid_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)


 Accuracy:  0.8571428571428571

Classification Report

               precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.67      1.00      0.80         2

    accuracy                           0.86         7
   macro avg       0.83      0.90      0.84         7
weighted avg       0.90      0.86      0.86         7



### Test tf-idf

In [14]:
# Load the previous dataset for testing
test = pd.read_csv("df_to_label.csv")
test.rename(columns={"label":"sentiment"}, inplace=True)
test["sentiment"] = None

In [15]:
# Select only new rows and not the training set
test_df = test.loc[~test.index.isin(train.index)]

In [16]:
# Predict on new values
test = test_df["text_wop_clean"].values
xtest_tfv = tfv.transform(test)
y_test = clf.predict(xtest_tfv)

In [17]:
# Add prediction to df
test_df["sentiment"] = y_test

In [25]:
# Display, seems to work decently
pd.set_option('display.max_colwidth', None) 
test_df[["text", "sentiment"]].head(100)

text  \
68                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             ROTTERDAM — Een grensconflict' tussen Nederland en België is er de oorzaak van dat kapitein Stelian Georg

### Cosine similarity between docs

In [56]:
# from: NLP with Python
def cosine_similarity(vector1, vector2):
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    return np.dot(vector1, vector2) / (np.sqrt(np.sum(vector1**2)) * np.sqrt(np.sum(vector2**2)))

In [59]:
for i in range(xtrain_tfv.shape[0]):
    for j in range(i + 1, xtrain_tfv.shape[0]):
        print("The cosine similarity between the documents ", i, "and", j, "is: ",
              cosine_similarity(xtrain_tfv.toarray()[i], xtrain_tfv.toarray()[j]))

The cosine similarity between the documents  0 and 1 is:  0.10040792993559246
The cosine similarity between the documents  0 and 2 is:  0.09332500725645572
The cosine similarity between the documents  0 and 3 is:  0.05696359526200015
The cosine similarity between the documents  0 and 4 is:  0.13151726416435253
The cosine similarity between the documents  0 and 5 is:  0.08311825591513892
The cosine similarity between the documents  0 and 6 is:  0.04750415870774301
The cosine similarity between the documents  0 and 7 is:  0.0290844684794286
The cosine similarity between the documents  0 and 8 is:  0.032824622882650735
The cosine similarity between the documents  0 and 9 is:  0.03696927805313887
The cosine similarity between the documents  0 and 10 is:  0.11074155224190163
The cosine similarity between the documents  0 and 11 is:  0.12377903875605836
The cosine similarity between the documents  0 and 12 is:  0.027812904512332405
The cosine similarity between the documents  0 and 13 is:  0

### TFIDF - Class

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import warnings
import pandas as pd


def Vectorize(vec, X_train, X_test):

    X_train_vec = vec.fit_transform(X_train)
    X_test_vec = vec.transform(X_test)

    print("Vectorization complete.\n")

    return X_train_vec, X_test_vec


# Use multiple classifiers and grid search for prediction
def ML_modeling(models, params, X_train, X_test, y_train, y_test):

    if not set(models.keys()).issubset(set(params.keys())):
        raise ValueError("Some estimators are missing parameters")

    for key in models.keys():

        model = models[key]
        param = params[key]
        gs = GridSearchCV(model, param, cv=10, error_score=0, refit=True)
        gs.fit(X_train, y_train)
        y_pred = gs.predict(X_test)

        # Print scores for the classifier
        print(key, ":", gs.best_params_)
        print(
            "Accuracy: %1.3f \tPrecision: %1.3f \tRecall: %1.3f \t\tF1: %1.3f\n"
            % (
                accuracy_score(y_test, y_pred),
                precision_score(y_test, y_pred, average="macro"),
                recall_score(y_test, y_pred, average="macro"),
                f1_score(y_test, y_pred, average="macro"),
            )
        )

    return


## Preparing to make a pipeline
models = {
    "Naive Bayes": MultinomialNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
}

params = {
    "Naive Bayes": {"alpha": [0.5, 1], "fit_prior": [True, False]},
    "Gradient Boosting": {"learning_rate": [0.05, 0.01], "min_samples_split": [2, 5]},
}

In [35]:
# Train-test split and vectorize
X_train, X_test, y_train, y_test = train_test_split(
    train["text_wop"], train["sentiment"], test_size=0.3, shuffle=True
)

X_train_vec, X_test_vec = Vectorize(TfidfVectorizer(min_df=3, max_features=None,
                      strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
                      ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1), X_train, X_test)

ML_modeling(models, params, X_train_vec, X_test_vec, y_train, y_test)

Vectorization complete.

Naive Bayes : {'alpha': 1, 'fit_prior': True}
Accuracy: 0.905 	Precision: 0.889 	Recall: 0.929 		F1: 0.899

Gradient Boosting : {'learning_rate': 0.01, 'min_samples_split': 2}
Accuracy: 0.762 	Precision: 0.757 	Recall: 0.679 		F1: 0.692

